<div style="border-style: solid;border-width: 1px;">
<div style="height: 200px; 
            width: 750px; 
            color: #ffffff;
            margin: auto;
	        background-color: #ff6600; 
	        background-image: url(http://icarus.lngs.infn.it/img/n3.jpg); 
            background-repeat: no-repeat;
            background-position: center;">
</div>

<div style="text-align:center">
    <h1><font color="blue"><font size="6">Analysis of Noise Filtering Techniques in ICARUS</font></font></h1><br>
    <font color="gray"><font size="3">A jupyter notebook aimed at studying Noise Filtering with the ICARUS LAr TPC</font></font><br>
</div>
</div>
<br>
<h3>What this notebook intends to do:</h3>
<ul>
    <li>Operates on art root format data files that contain raw::RawDigit objects</li>
        <ul>
            <li>The RawDigits contain waveforms representing the TPC readout </li>
        </ul>
    <li>Once the RawDigits have been recovered the idea is to analyze the raw data</li>
        <ul>
            <li>Understand impacts of signal degradation on coherent noise removal</li>
            <li>Look at various algorithms for reducing white noise while maintaining signal</li>
        </ul>
</ul>

<h3>What is needed to run this notebook:</h3>
<ul>
    <li><p>Necessary python libraries/includes</p>
        <ul>
            <li> matplotlib   - the basic plotting package though note that here we mostly use plot-ly</li>
            <li> numpy        - python array handling and loads of useful functions</li>
            <li> scipy        - where we get the analytic functions we will make heavy use of </li>
            <li> ipywidgets   - useful for making interactive output</li>
            <li> uproot       - see below</li>
            <li> plotly       - the plot-ly graphing package which allows interactive plots</li>
        </ul></li>
    <li>uproot<br>
        In this notebook we use "uproot" to handle the root input. For more information on uproot follow this <a href="https://uproot.readthedocs.io/en/latest/">link</a></li>
</ul>



<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Start Up Block</font></font></h2><br>
</div>

<ul>
    <li>Import of standard libraries used below</li>
    <li>Set the system path to include looking in the package of noise handling functions/objects</li>
    <li>We define two variables which define the location of the input file</li>
        <ul>
            <li> PATHNAME       - Fully qualified path to the folder containing the input file</li>
            <li> RECOFILENAME   - the fully qualified file name for the input data file</li>
        </ul>
    <li>With these definitions we can open and read the input data file</li>
</ul>



In [3]:
# Probably most people will simply want "matplotlib inline" but on a mac it is useful to do the other stuff
#%matplotlib inline
##matplotlib qt
##matplotlib notebook
##config InlineBackend.figure_format ='retina'
#%pylab

# This allows reading of root files without having to actually use root. Nice!
import uproot

# Widgets are always good for you
#from ipywidgets import widgets,Layout

###############################################################################
# TODO:
# Set the path to our library of functions/objects/etc
import sys

sigProcPath = "/home/usher/LArTPC/jupyter/icarus-sigproc-tools"
sys.path.insert(0,sigProcPath)
###############################################################################
# TODO:
# Set the path and name of the data file to be read
PATHNAME       = "/media/usher/analysis/LArSoft/ICARUS/workarea/data"
#RECOFILENAME   = PATHNAME + "/data_run621_1_EW05_PED_20190926T201608_DECODE.root"data_dl1_run1154_11_20200220T055954_dl3.root
#RECOFILENAME   = PATHNAME + "/data_run618_2_EW01M_PED_20190926T174904_DECODE.root"
#RECOFILENAME   = PATHNAME + "/daqTestJan/data_dl1_run822_201_20200109T010140_dl3_20200118T180439-TPCANALYSIS.root"
#RECOFILENAME   = PATHNAME + "/daqTestJan/data_dl2_run822_251_20200109T013500_dl4_20200121T025820-DECODER.root"
#RECOFILENAME   = PATHNAME + "/daqTestFeb/data_dl1_run1154_10_20200220T055911_dl3_20200229T035706-decoder.root"
RECOFILENAME   = PATHNAME + "/daqTestFeb/data_dl1_run1152_11_20200220T001445_dl3.root"

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Open and Read the Input File</font></font></h2><br>
</div>

<ul>
    <li>We define three variables which should be common to all files
        <ul>
            <li> RECOFOLDERNAME - the name of the folder for the particular data set we want to analyze</li>
            <li> DAQNAME.       - currently unused</li>
        </ul></li>
    <li>With these definitions we can open and read the input data file</li>
    <li>Contents of the input file are read into a numpy array</li>
</ul>



In [4]:
# Below should be standard for the test data files currently available
RECOFOLDERNAME = "Events" 
#DAQNAME        = "raw::RawDigits_daq__TPCANALYSIS."
#DAQNAME        = "raw::RawDigits_daq__DECODER."
#DAQNAME        = "raw::RawDigits_daq__decoder."
DAQNAME        = "raw::RawDigits_daq_RAW_decoder."

# Grab the pandas dataframe from the input file for the tree we want to look at
print("Opening file: ",RECOFILENAME)
data_file = uproot.open(RECOFILENAME)

print(data_file.keys(),"\n------\n")

print("Opening the folder contianing the RawDigits information: ",RECOFOLDERNAME)
events_folder = data_file[RECOFOLDERNAME]

print(events_folder.keys())


Opening file:  /media/usher/analysis/LArSoft/ICARUS/workarea/data/daqTestFeb/data_dl1_run1152_11_20200220T001445_dl3.root
['RootFileDB;1', 'MetaData;1', 'FileIndex;1', 'Parentage;1', 'EventHistory;1', 'Events;1', 'EventMetaData;1', 'SubRuns;1', 'SubRunMetaData;1', 'Runs;1', 'RunMetaData;1', 'ResultsTree;1', 'ResultsMetaData;1'] 
------

Opening the folder contianing the RawDigits information:  Events
['EventAuxiliary', 'artdaq::Fragments_daq_BERNCRTZMQ_DAQ.', 'artdaq::Fragments_daq_BERNCRTZMQ_DAQ./artdaq::Fragments_daq_BERNCRTZMQ_DAQ.present', 'artdaq::Fragments_daq_BERNCRTZMQ_DAQ./artdaq::Fragments_daq_BERNCRTZMQ_DAQ.rangeSetID', 'artdaq::Fragments_daq_BERNCRTZMQ_DAQ./artdaq::Fragments_daq_BERNCRTZMQ_DAQ.obj', 'artdaq::Fragments_daq_BERNCRTZMQ_DAQ./artdaq::Fragments_daq_BERNCRTZMQ_DAQ.obj/artdaq::Fragments_daq_BERNCRTZMQ_DAQ.obj.vals_.size_', 'artdaq::Fragments_daq_BERNCRTZMQ_DAQ./artdaq::Fragments_daq_BERNCRTZMQ_DAQ.obj/artdaq::Fragments_daq_BERNCRTZMQ_DAQ.obj.vals_.data_', 'artdaq::

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Set Up and Run the Noise Filtering Object</font></font></h2><br>
</div>
<p>Here we instantiate our noise filtering object. We pass in the array of RawDigit events to get set up and then call the routine to do the filtering of the events</p>

In [5]:
import numpy as np
from sigproc_tools.sigproc_objects.filtereventsbycrate import FilterEventsByCrate
#from sigproc_tools.sigproc_objects.filterevents import FilterEvents

numChannelsPerCrate = 576
crateNum            = 1

filter = FilterEventsByCrate(events_folder,DAQNAME,crateNum)
#filter = FilterEvents(events_folder,DAQNAME)

# What are the basic operating parameters?
nChannelsPerGroup = 32   # <==== We choose this
numChannels       = filter.rawdigits.numChannels(0)
numCratesThisData = numChannels // numChannelsPerCrate
numGroups         = numChannels // nChannelsPerGroup // numCratesThisData

print("Number channels:",numChannels,", Group:",nChannelsPerGroup,", # crates:",numCratesThisData,", numGroups:",numGroups)

numEvents = filter.filterEvents(nChannelsPerGroup)

print("Noise processing complete with",numEvents)

AttributeError: 'Model_TTree_v20' object has no attribute 'array'

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Basic Set Up for Visualization of Items of Interest</font></font></h2><br>
</div>
<br>
<p>We want to visualize some basic stuff so recover it and get averages over the number of events</p>

In [ ]:
# Get average pedestal for the events
print("Get average of pedestals and rms")

rawPedestals = np.mean([filter.getRawDigits().getPedestals(idx) for idx in range(filter.numEvents)],axis=0)[filter.startIndex:filter.stopIndex]
avePedestals = np.mean(filter.pedestalsAll,axis=0)
rawRMSVals   = np.mean([filter.getRawDigits().getRMSVals(idx) for idx in range(filter.numEvents)],axis=0)[filter.startIndex:filter.stopIndex]
aveRMS       = np.mean(filter.rmsAll,axis=0)

print("Getting average of intrinsic noise")

aveIntrinsicRMSEvent = np.mean(filter.intrinsicRMSAll,axis=0)

# compute the channel by channel rms after coherent noise subtraction
print("Now do RMS after coherent noise removal")

channelRMSAll = np.sqrt(np.mean(np.square(filter.waveLessCoherentAll),axis=2))
channelRMS    = np.mean(channelRMSAll,axis=0)

print("Mission accomplished")

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Output a First Set of Basic Plots</font></font></h2><br>
</div>

<ul>
    <li><p>Get a first look at some basic quantities</p>
        <ul>
            <li> The "raw" pedestals on a channel by channel basis </li>
            <li> The "raw" RMS on a channel by channel basis</li>
            <li> The average of the RMS computed for each tick in a group of waveforms</li>
            <ul>
                <li> Note that we consider this to be the "intrinsic" noise - meaning essentially just the noise due to the cables and electronics but now with other effects removed</li>
            </ul>
        </ul></li>
</ul>
<p><b>NOTE:</b> the plots that are output are interactive, you can click and drag to resize them, zoom, etc. </p>



In [ ]:
import plotly.graph_objects as go
import plotly.subplots as subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS

channelRange = np.arange(filter.rawdigits.numChannels(0))
tickRange    = np.arange(filter.rawdigits.numTicks(0))

#print(rawPedestals,", ",rawPedestals.shape)
#print(filter.getRawDigits().getPedestals(0)[filter.startIndex:filter.stopIndex])
#print(avePedestals,", ",avePedestals.shape)

#aveIntrinsicRMS = np.mean(intrinsicRMS,axis=0)
aveIntrinsicRMS = np.mean(aveIntrinsicRMSEvent,axis=0)

print(aveIntrinsicRMS,", ",aveIntrinsicRMS.shape)

subFig = subplots.make_subplots(rows=4,cols=1)
subFig.add_trace(go.Scatter(x=channelRange,y=rawPedestals,name="Pedestals",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)),row=1,col=1)
subFig.add_trace(go.Scatter(x=channelRange,y=avePedestals,name="Pedestals",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1)),row=1,col=1)
subFig.add_trace(go.Scatter(x=channelRange,y=rawRMSVals,name="Raw RMS",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)),row=2,col=1)
subFig.add_trace(go.Scatter(x=channelRange,y=aveRMS,name="Total RMS",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1)),row=2,col=1)
subFig.add_trace(go.Scatter(x=channelRange,y=channelRMS,name="Clean RMS",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1)),row=3,col=1)
subFig.add_trace(go.Scatter(x=tickRange,y=aveIntrinsicRMS,name="Intrinsic RMS",line=dict(color=DEFAULT_PLOTLY_COLORS[3],width=1)),row=4,col=1)

subFig.update_layout(
        width=800,
        height=800,
        autosize=False
    )


subFig.show()

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Make a quick 3D plot of this datat</font></font></h2><br>
</div>



In [ ]:
from plotting.graphs import *

makePlots = True

if makePlots:
    #overlayPic = plotEventView3D(np.transpose(fullEvent))
    #overlayPic = plotEventView3D(np.transpose(filter.waveLessPedAll[0,:,:]))
    overlayPic = plotEventView3D(np.transpose(filter.waveLessCoherentAll[0,:,:]))
    #overlayPic = plotEventView3D(np.transpose(filter.rawWaveforms[0,:,:]))
    print("Calling the show method")
    overlayPic.show()


<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Histogram and the Fit the Median Values RMS by tick Plot</font></font></h2><br>
</div>
<p> We can try to determine the value of the intrinsic noise by turning the bottom plot above into a histogram and fitting it. </p> 

In [ ]:
# Set up for fitting
from scipy.optimize import curve_fit

#save_error_state  = np.seterr(divide='ignore')
#np.seterr(**save_error_state)
import plotly.tools as tls

# Define model function to be used to fit to the data above:
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

# Start with the median waveforms since we have more work to do here
#medianRMSVals = np.sqrt(np.mean(np.square(medianAll),axis=2))

nBins  = 20
lowBin = 2.23 #4.5 #2.45 #2.42 #3.45    #2.37
hiBin  = 2.43 #7.5 #2.65 #2.54 #3.70   #2.47
stepSize = (hiBin-lowBin)/nBins
xBins = np.arange(lowBin,hiBin,stepSize)

# Basic idea here is to fill a numpy histogram and perform the fit on that
#hist,_ = np.histogram(aveIntrinsicRMS,bins=xBins.size,range=(xBins[0],xBins[-1]))
hist,_ = np.histogram(aveIntrinsicRMS,bins=xBins.size,range=(xBins[0],xBins[-1]))

fitArray = hist.astype(np.float64)

#max bin?
maxValue = np.amax(hist)
maxBin   = np.where(hist == maxValue)
maxRMS   = xBins[maxBin]

print("hist:",hist)
print("max RMS: ",maxRMS)

fitParams = np.array([maxValue,maxRMS,4*stepSize]).astype(np.float64)

# here is where the fit is done
coeff,varMatrix = curve_fit(gauss,xBins.astype(np.float64),fitArray,p0=fitParams)

# Apparently, if we want to use plotly (sigh... we do) then we need to create an occurance vector...
# So, here goes! 
histoArray = np.empty([1])

for idx in range(xBins.size):
    if fitArray[idx] > 0.:
        repeatArr  = np.repeat(xBins[idx],fitArray[idx].astype(int))
        histoArray = np.concatenate((histoArray,repeatArr),axis=0)

print("Fit coefficients - area: ",coeff[0],", mean: ",coeff[1],", sigma: ",coeff[2])

fitCurve = gauss(xBins,*np.array(coeff))

firstBinCenter = lowBin + 0.5 * stepSize
lastBinCenter  = hiBin + 0.5 * stepSize

fitPlot = go.Figure()

fitPlot.add_trace(go.Histogram(x=histoArray[1:],autobinx=False,xbins=dict(start=firstBinCenter,end=lastBinCenter),nbinsx=nBins,name="RMS values"))
fitPlot.add_trace(go.Scatter(x=xBins,y=fitCurve,name="Fit to RMS",line=dict(color="red",width=1)))

fitPlot.show()

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Plot the "Waveforms" of the Median Value Coherent Noise Correction</font></font></h2><br>
</div>

<p> Here we plot the median correction for each tick and separately for each grouping </p>


In [ ]:
#Plot the median

numColors = len(DEFAULT_PLOTLY_COLORS)

fig = go.Figure()

xRange = np.arange(filter.medianAll.shape[2])

for idx in range(filter.medianAll.shape[1]):
    fig.add_trace(go.Scatter(x=xRange,y=filter.medianAll[1,idx,:],name="Median "+str(idx),
                             line=dict(color=DEFAULT_PLOTLY_COLORS[idx%numColors],width=1)))

fig.show()



<p>It is interesting to note in the above (particularly if you zoom in) that there appear to be remaining correlations between pairs of groupings</p>
<p><b>This needs to be investigated!</b></p>

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Compute and Display the Correlations of the Coherent Noise</font></font></h2><br>
</div>
<p> This is currently using the Pearson R test and cross correlation </p>

In [ ]:
from sigproc_tools.sigproc_functions.noiseAnalysis import computeCorrelations

#numGroups = 9

print("Inputting",numEvents," events and",numGroups," groups")

# Call the function to compute the correlation matrices
statCorMatrix,xcorCorMatrix = computeCorrelations(filter.medianAll,numEvents,numGroups)

# And now make the flashy plot-ly plots
corFig = subplots.make_subplots(rows=2,cols=1,subplot_titles=("Pearson R","Cross Correlation"),vertical_spacing=0.12)

corFig.add_trace(go.Heatmap(z=statCorMatrix, colorscale="Viridis",colorbar=dict(len=0.42,y=0.79)),row=1,col=1)
corFig.add_trace(go.Heatmap(z=xcorCorMatrix, colorscale="Viridis",colorbar=dict(len=0.42,y=0.21)),row=2,col=1)

corFig.update_layout(height=800,width=800,title_text="Correlation Maps",title_font_size=30)
corFig.update_xaxes(title_text="Grouping",row=1,col=1)
corFig.update_xaxes(title_text="Grouping",row=2,col=1)
corFig.update_yaxes(title_text="Grouping",row=1,col=1)
corFig.update_yaxes(title_text="Grouping",row=2,col=1)

corFig.show()

In [ ]:
# Call the function to compute the correlation matrices
# waveLessCoherentAll for coherent noise corrected
# waveLessPedAll for pedestal corrected
statCorMatrix,xcorCorMatrix = computeCorrelations(filter.waveLessPedAll[:,0:nChannelsPerGroup,:],numEvents,nChannelsPerGroup)

# And now make the flashy plot-ly plots
corWaveFig = subplots.make_subplots(rows=2,cols=1,subplot_titles=("Pearson R","Cross Correlation"),vertical_spacing=0.12)

corWaveFig.add_trace(go.Heatmap(z=statCorMatrix, colorscale="Viridis",colorbar=dict(len=0.42,y=0.79)),row=1,col=1)
corWaveFig.add_trace(go.Heatmap(z=xcorCorMatrix, colorscale="Viridis",colorbar=dict(len=0.42,y=0.21)),row=2,col=1)

corWaveFig.update_layout(height=800,width=800,title_text="Correlation Maps",title_font_size=30)
corWaveFig.update_xaxes(title_text="Grouping",row=1,col=1)
corWaveFig.update_xaxes(title_text="Grouping",row=2,col=1)
corWaveFig.update_yaxes(title_text="Grouping",row=1,col=1)
corWaveFig.update_yaxes(title_text="Grouping",row=2,col=1)

corWaveFig.show()

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Compute the Power Spectrum of the Coherent Noise Correction</font></font></h2><br>
</div>

<p>First on the list to study is the noise spectrum due to the coherent noise across a motherboard. We can look at this by taking the FFT of the median "waveforms" plotted above and then computing the power spectrum. At that point we can average across all of the mother board groups to try to see what stands out. </p>



In [ ]:
import scipy.signal as signal
import scipy.fft    as fft

numColors = len(DEFAULT_PLOTLY_COLORS)

#Time to look at FFT
icarusTickWid = 0.4                # 0.4 microseconds/tick
icarusMaxFreq = 1. / icarusTickWid # max frequency in MHz
motherboard   = 8

# To do the FFT we want to remove the pedestal completely
# Note: this was originally put in becasue I had erroneously fed the wrong waveform to the coherent noise
# subtraction, so this is probably no longer an issue. But will leave for completeness
meanValue = np.mean(filter.medianAll,axis=2)

corWave = filter.medianAll - meanValue.reshape((meanValue.shape[0],meanValue.shape[1],1))

# Use scipy to compute the power spectrum directly
freqVec, medianPow = signal.periodogram(corWave,icarusMaxFreq,axis=2)
#freqVec, medianPow = signal.periodogram(corWave[0,:,:],icarusMaxFreq,axis=1)

corWaveFFT = fft.rfft(x=corWave,axis=2)
#corWaveFFT = fft.rfft(x=corWave[0,:,:],axis=1)

numTimeBins = corWaveFFT.shape[2]
#numTimeBins = corWaveFFT.shape[1] #[2]
numFreqBins = numTimeBins//2 + 1

print("Number of time bins:",numTimeBins,", number frequency bins:",numFreqBins)

absCorWaveFFT  = np.absolute(corWaveFFT[:,:,:numFreqBins])
#absCorWaveFFT  = np.absolute(corWaveFFT[:,:numFreqBins])
powerSpectrum  = icarusTickWid * np.square(np.absolute(corWaveFFT[:,:,:numFreqBins])) / numFreqBins
#powerSpectrum  = icarusTickWid * np.square(np.absolute(corWaveFFT)) / numFreqBins
halfSpectrum   = powerSpectrum / 2
backWards      = np.sqrt(numFreqBins * powerSpectrum / icarusTickWid)
scaledSpectrum = np.multiply(powerSpectrum,halfSpectrum)

transformBack  = fft.irfft(corWaveFFT,axis=2) #backWards,axis=2)
#transformBack  = fft.irfft(corWaveFFT,axis=1) #backWards,axis=2)

print("shape of corWaveFFT:",corWaveFFT.shape,", power spectrum shape:",powerSpectrum.shape)
print("The input:",corWave[0,10,0:9])
print("A few elements:",corWaveFFT[0,10,0:9])
#print("A few elements:",corWaveFFT[10,0:9])
print("  absolute(corWaveFFT):",absCorWaveFFT[0,10,0:9])
#print("  absolute(corWaveFFT):",absCorWaveFFT[10,0:9])
print(" and of power:",powerSpectrum[0,10,0:9])
#print(" and of power:",powerSpectrum[10,0:9])
#print(" half:",halfSpectrum[0,10,0:9])
#print(" and even more:",scaledSpectrum[0,10,0:9])
#print(" backwards:",backWards[0,10,0:9])
print(" backwards:",backWards[10,0:9])
print(" transform back:",transformBack[0,10,0:9])
#print(" transform back:",transformBack[10,0:9])

# Try to get the median filter 
medianFilt = signal.medfilt(medianPow,[1,1,21])
#medianFilt = signal.medfilt(medianPow,[1,21])

print("Shape of power:",medianPow.shape,", shape of median:",medianFilt.shape)

# combine the power
meanPower = np.mean(np.mean(medianPow,axis=1),axis=0)
meanPower = np.mean(medianPow,axis=0)
meanFilt  = np.mean(medianFilt,axis=0)
meanSpect = np.mean(powerSpectrum,axis=0)
#meanPower = medianPow
#meanFilt  = medianFilt
#meanSpect = powerSpectrum

#figFFT = subplots.make_subplots(rows=3,cols=1,subplot_titles=("Low Channels","High Channels","Shared"),vertical_spacing=0.12)

figFFT = subplots.make_subplots(rows=1,cols=1)


for idx in range(numGroups):
    figFFT.add_trace(go.Scatter(x=freqVec,y=meanPower[idx],name="Power "+str(idx),line=dict(color=DEFAULT_PLOTLY_COLORS[idx%numColors],width=1)),row=1,col=1)
#    figFFT.add_trace(go.Scatter(x=freqVec,y=meanFilt[idx],name="Filter "+str(idx),line=dict(color=DEFAULT_PLOTLY_COLORS[idx%numColors],width=1)),row=1,col=1)
#    figFFT.add_trace(go.Scatter(x=freqVec,y=meanSpect[idx],name="Spectrum "+str(idx),line=dict(color=DEFAULT_PLOTLY_COLORS[idx%numColors],width=2)),row=1,col=1)

#    figFFT.add_trace(go.Scatter(x=freqVec,y=meanPower[idx],name="Low "+str(idx),
#                             line=dict(color=DEFAULT_PLOTLY_COLORS[idx%numColors],width=1)),row=1,col=1)
#    figFFT.add_trace(go.Scatter(x=freqVec,y=meanPower[idx+9],name="High "+str(idx+9),
#                             line=dict(color=DEFAULT_PLOTLY_COLORS[idx%numColors],width=2)),row=2,col=1)
#
#figFFT.add_trace(go.Scatter(x=freqVec,y=meanPower[8],name="Shared "+str(8),
#                             line=dict(color=DEFAULT_PLOTLY_COLORS[8%numColors],width=2)),row=3,col=1)


#figFFT.add_trace(go.Scatter(x=freqVec,y=meanPower[17],name="Median FFT",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)))
figFFT.update_layout(height=800,width=1000,title_text="Power Spectra averaged over all events",title_font_size=30)

figFFT.show()



In [ ]:
# Use scipy to compute the power spectrum of each waveform for each event
allFreqVec, allWavePower = signal.periodogram(filter.waveLessPedAll,icarusMaxFreq,axis=2)

# combine the power
#meanPower = np.mean(np.mean(medianPow,axis=1),axis=0)
allMeanPower = np.mean(allWavePower,axis=0)

In [ ]:
from plotting.graphs import plotEventView2D

eventPic = plotEventView2D(allMeanPower)
